In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Run query

In [6]:
import sys
sys.path.append('../SelectionFunctionToolkit')

In [ ]:
from SelectionFunctionGaiaArchive import GaiaArchiveQuery

mag_res=0.1; col_res=0.1; pos_level=7

version="gaiaedr3"; mag_res=mag_res; col_res=col_res; pos_level=pos_level
query = GaiaArchiveQuery(magnitude_column = 'phot_g_mean_mag',
                         magnitude_resolution = mag_res,
                         colour_column = 'bp_rp',
                         colour_resolution = col_res,
                         position_column='source_id',
                         position_resolution=pos_level,
                         gaia_version=version,
                         superset_criteria = ['phot_g_mean_mag is not NULL', 'phot_g_mean_mag>2', 'phot_g_mean_mag<23'],
                         subset_criteria=['ruwe < 1.4', 'astrometric_params_solved>3'])

In [12]:
print(query.query)

select magnitude, colour, position, count(*) as n, sum(selection) as k 
 from ( select  to_integer(floor(phot_g_mean_mag * 10)) as magnitude, 
to_integer(floor(bp_rp * 2)) as colour, 
gaia_healpix_index(7, source_id) as position, 
to_integer(IF_THEN_ELSE(ruwe < 1.4, 1.0, 0.0)) as selection from gaiaedr3.gaia_source 
 where phot_g_mean_mag>-5 and phot_rp_mean_mag < 30  ) as subquery 
 group by magnitude, colour, position


In [13]:
%time query.run_query()

Running query
500 Error 500:
null


HTTPError: Error 500:
null

In [ ]:
with h5py.File(f'/data/asfe2/Projects/astrometry/gaiaedr3_ruwe1p4_kncounts.h', 'w') as hf:
    for key in ['magnitude', 'colour', 'position', 'k', 'n']:
        hf.create_dataset(key, data=np.array(query.results[key]))
    
    hf.create_dataset('magnitude_res', data=np.array([mag_res]))
    hf.create_dataset('colour_res', data=np.array([col_res]))
    hf.create_dataset('position_level', data=np.array([pos_level]))
    hf.attrs["query"] = query.query